In [1]:
from scipy.spatial.distance import pdist, cdist, squareform
import math
import pandas as pd
import numpy as np
from numba import cuda
from numba import jit
from sklearn import datasets
import cupy as cp

In [2]:
def grid_set(data, N):

    _ , W = data.shape
    AvD1 = data.mean(0)
    X1 = cp.mean(cp.sum(cp.power(data,2),axis=1))
    grid_trad = cp.sqrt(2*(X1 - AvD1*AvD1.T))/N
    Xnorm = cp.sqrt(cp.sum(cp.power(data,2),axis=1))
    aux = Xnorm
    for i in range(W-1):
        aux = cp.insert(aux,0,Xnorm.T,axis=1)
    data = data / aux
    seq = cp.argwhere(cp.isnan(data))
    if tuple(seq[::]): data[tuple(seq[::])] = 1
    AvD2 = data.mean(0)
    grid_angl = cp.sqrt(1-AvD2*AvD2.T)/N
    return X1, AvD1, AvD2, grid_trad, grid_angl

In [3]:
def pi_calculator(Uniquesample, mode):
    UN, W = Uniquesample.shape
    if mode == 'euclidean' or mode == 'mahalanobis' or mode == 'cityblock' or mode == 'chebyshev' or mode == 'canberra':
        AA1 = Uniquesample.mean(0)
        X1 = sum(sum(cp.power(Uniquesample,2)))/UN
        DT1 = X1 - sum(cp.power(AA1,2))
        aux = []
        for i in range(UN): aux.append(AA1)
        
        #aux2 = [Uniquesample[i]-aux[i] for i in range(UN)]
        #uspi = cp.sum(cp.power(aux2,2),axis=1)+DT1
        
        uspi = cp.power(cdist(Uniquesample, aux, mode),2)+DT1
        uspi = uspi[:,0]
        
    if mode == 'minkowski':
        AA1 = Uniquesample.mean(0)
        X1 = sum(sum(cp.power(Uniquesample,2)))/UN
        DT1 = X1 - sum(cp.power(AA1,2))
        aux = cp.matrix(AA1)
        for i in range(UN-1): aux = cp.insert(aux,0,AA1,axis=0)
        aux = cp.array(aux)
        
        uspi = cp.power(cdist(Uniquesample, aux, mode, p=1.5),2)+DT1
        uspi = uspi[:,0]
    
    if mode == 'cosine':
        Xnorm = cp.matrix(cp.sqrt(cp.sum(cp.power(Uniquesample,2),axis=1))).T
        aux2 = Xnorm
        for i in range(W-1):
            aux2 = cp.insert(aux2,0,Xnorm.T,axis=1)
        Uniquesample1 = Uniquesample / aux2
        AA2 = cp.mean(Uniquesample1,0)
        X2 = 1
        DT2 = X2 - cp.sum(cp.power(AA2,2))
        aux = []
        for i in range(UN): aux.append(AA2)
        aux2 = [Uniquesample1[i]-aux[i] for i in range(UN)]
        uspi = cp.sum(cp.sum(cp.power(aux2,2),axis=1),axis=1)+DT2
        
    return uspi

In [4]:
def Globaldensity_Calculator(data, distancetype):
    Uniquesample, J, K = cp.unique(data, axis=0, return_index=True, return_inverse=True)
    Frequency, _ = cp.histogram(K,bins=len(J))
    uspi1 = pi_calculator(Uniquesample, distancetype)
    sum_uspi1 = sum(uspi1)
    Density_1 = uspi1 / sum_uspi1
    uspi2 = pi_calculator(Uniquesample, 'cosine')
    sum_uspi2 = sum(uspi2)
    Density_2 = uspi1 / sum_uspi2
    GD = (Density_2+Density_1) * Frequency
    index = GD.argsort()[::-1]
    GD = GD[index]
    Uniquesample = Uniquesample[index]
    Frequency = Frequency[index]
    return GD, Uniquesample, Frequency

In [5]:
def chessboard_division(Uniquesample, MMtypicality, interval1, interval2, distancetype):
    L, W = Uniquesample.shape
    if distancetype == 'euclidean':
        W = 1
    BOX = [Uniquesample[k] for k in range(W)]
    BOX_miu = [Uniquesample[k] for k in range(W)]
    BOX_S = [1]*W
    BOX_X = [sum(Uniquesample[k]**2) for k in range(W)]
    NB = W
    BOXMT = [MMtypicality[k] for k in range(W)]
    
    for i in range(W,L):
        if distancetype == 'minkowski':
            a = cdist(Uniquesample[i].reshape(1,-1), BOX_miu, metric=distancetype, p=1.5)
        else:
            a = cdist(Uniquesample[i].reshape(1,-1), BOX_miu, metric=distancetype)
        
        b = cp.sqrt(cdist(Uniquesample[i].reshape(1,-1), BOX_miu, metric='cosine'))
        distance = cp.array([a[0],b[0]]).T
        SQ = []
        for j,d in enumerate(distance):
            if d[0] < interval1 and d[1] < interval2:
                SQ.append(j)
        #SQ = cp.argwhere(distance[::,0]<interval1 and (distance[::,1]<interval2))
        COUNT = len(SQ)
        if COUNT == 0:
            BOX.append(Uniquesample[i])
            NB = NB + 1
            BOX_S.append(1)
            BOX_miu.append(Uniquesample[i])
            BOX_X.append(sum(Uniquesample[i]**2))
            BOXMT.append(MMtypicality[i])
        if COUNT >= 1:
            DIS = distance[SQ[::],0]/interval1 + distance[SQ[::],1]/interval2 # pylint: disable=E1136  # pylint/issues/3139
            b = cp.argmin(DIS)
            BOX_S[SQ[b]] = BOX_S[SQ[b]] + 1
            BOX_miu[SQ[b]] = (BOX_S[SQ[b]]-1)/BOX_S[SQ[b]]*BOX_miu[SQ[b]] + Uniquesample[i]/BOX_S[SQ[b]]
            BOX_X[SQ[b]] = (BOX_S[SQ[b]]-1)/BOX_S[SQ[b]]*BOX_X[SQ[b]] + sum(Uniquesample[i]**2)/BOX_S[SQ[b]]
            BOXMT[SQ[b]] = BOXMT[SQ[b]] + MMtypicality[i]
    return BOX, BOX_miu, BOX_X, BOX_S, BOXMT, NB

In [6]:
def ChessBoard_PeakIdentification(BOX_miu,BOXMT,NB,Internval1,Internval2, distancetype):
    Centers = []
    n = 2
    ModeNumber = 0
    
    if distancetype == 'minkowski':
        distance1 = squareform(pdist(BOX_miu,metric=distancetype, p=1.5))
    else:
        distance1 = squareform(pdist(BOX_miu,metric=distancetype))        
    
    distance2 = cp.sqrt(squareform(pdist(BOX_miu,metric='cosine')))
    for i in range(NB):
        seq = []
        for j,(d1,d2) in enumerate(zip(distance1[i],distance2[i])):
            if d1 < n*Internval1 and d2 < n*Internval2:
                seq.append(j)
        Chessblocak_typicality = [BOXMT[j] for j in seq]
        if max(Chessblocak_typicality) == BOXMT[i]:
            Centers.append(BOX_miu[i])
            ModeNumber = ModeNumber + 1
    return Centers, ModeNumber

In [7]:
def cloud_member_recruitment(ModelNumber,Center_samples,Uniquesample,grid_trad,grid_angl, distancetype):
    L, W = Uniquesample.shape
    Membership = cp.zeros((L,ModelNumber))
    Members = cp.zeros((L,ModelNumber*W))
    Count = []

    if distancetype == 'minkowski':
        distance1 = cdist(Uniquesample,Center_samples, metric=distancetype, p=1.5)/grid_trad
    else:
        distance1 = cdist(Uniquesample,Center_samples, metric=distancetype)/grid_trad

    distance2 = cp.sqrt(cdist(Uniquesample, Center_samples, metric='cosine'))/grid_angl
    distance3 = distance1 + distance2
    B = distance3.argmin(1)
    for i in range(ModelNumber):
        seq = []
        for j,b in enumerate(B):
            if b == i:
                seq.append(j)
        Count.append(len(seq))
        Membership[:Count[i]:,i] = seq
        Members[:Count[i]:,W*i:W*(i+1)] = [Uniquesample[j] for j in seq]
    MemberNumber = Count
    return Members,MemberNumber,Membership,B 

In [8]:
def data_standardization(data,X_global,mean_global,mean_global2,k):
    mean_global_new = k/(k+1)*mean_global+data/(k+1)
    X_global_new = k/(k+1)*X_global+cp.sum(cp.power(data,2))/(k+1)
    mean_global2_new = k/(k+1)*mean_global2+data/(k+1)/cp.sqrt(cp.sum(cp.power(data,2)))
    return X_global_new, mean_global_new, mean_global2_new

In [9]:
def Chessboard_online_division(data,Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    distance = cp.zeros((NB,2))
    COUNT = 0
    SQ = []
    for i in range(NB):
        distance[i,0] = pdist([list(BOX_miu[i]), data.tolist()[0]],'euclidean')
        distance[i,1] = cp.sqrt(pdist([list(BOX_miu[i]), data.tolist()[0]],'cosine'))
        if distance[i,0] < intervel1 and distance[i,1] < intervel2:
            COUNT += 1
            SQ.append(i)
            
    if COUNT == 0:
        Box_new = cp.concatenate((Box, cp.array(data)))
        NB_new = NB+1
        BOX_S_new = cp.concatenate((BOX_S, cp.array([1])))
        #BOX_S_new = cp.array(BOX_S)
        BOX_miu_new = cp.concatenate((BOX_miu, cp.array(data)))
    if COUNT>=1:
        DIS = cp.zeros((COUNT,1))
        for j in range(COUNT):
            DIS[j] = distance[SQ[j],0] + distance[SQ[j],1]
        a = cp.amin(DIS)
        b = int(cp.where(DIS == a)[0])
        Box_new = Box
        NB_new = NB
        BOX_S_new = cp.array(BOX_S)
        BOX_miu_new = cp.array(BOX_miu)
        BOX_S_new[SQ[b]] = BOX_S[SQ[b]] + 1
        BOX_miu_new[SQ[b]] = BOX_S[SQ[b]]/BOX_S_new[SQ[b]]*BOX_miu[SQ[b]]+data/BOX_S_new[SQ[b]]
    
            
    
    return Box_new,BOX_miu_new,BOX_S_new,NB_new

In [10]:
def Chessboard_online_merge(Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    threshold1=intervel1/2
    threshold2=intervel2/2
    NB1=0
    
    while NB1 != NB:
        CC = 0
        NB1 = NB
        for ii in range(NB):
            seq1 = [i for i in range(NB) if i != ii]
            distance1 = cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'euclidean')
            distance2 = cp.sqrt(cdist(BOX_miu[ii].reshape(1,-1), BOX_miu[seq1], 'cosine'))
            for jj in range(NB-1):
                if distance1[0,jj] < threshold1 and distance2[0,jj] < threshold2:
                    CC = 1
                    NB -= 1
                    Box = cp.delete(Box, (ii), axis=0)
                    BOX_miu[seq1[jj]] = BOX_miu[seq1[jj]]*BOX_S[seq1[jj]]/(BOX_S[seq1[jj]]+BOX_S[ii])+BOX_miu[ii]*BOX_S[ii]/(BOX_S[seq1[jj]]+BOX_S[ii])
                    
                    BOX_S[seq1[jj]] = BOX_S[seq1[jj]] + BOX_S[ii]
                    BOX_miu = cp.delete(BOX_miu, (ii), axis=0)
                    BOX_S = cp.delete(BOX_S, (ii), axis=0)
                    
                    break
            if CC == 1:
                break
                    
    return Box,BOX_miu,BOX_S,NB

In [11]:
def Chessboard_globaldensity(Hypermean,HyperSupport,NH):
    uspi1 = pi_calculator(Hypermean,'euclidean')
    sum_uspi1 = cp.sum(uspi1)
    Density_1 = uspi1/sum_uspi1
    uspi2 = pi_calculator(Hypermean,'cosine')
    sum_uspi2 = cp.sum(uspi2)
    Density_2 = uspi1/sum_uspi2;
    Hyper_GD = (Density_2 + Density_1)*HyperSupport
    return Hyper_GD

In [12]:
def ChessBoard_online_projection(BOX_miu,BOXMT,NB,interval1,interval2):
    Centers = []
    ModeNumber = 0
    n = 2
    
    for ii in range(NB):
        Reference = BOX_miu[ii]
        distance1 = cp.zeros((NB,1))
        distance2 = cp.zeros((NB,1))
        for i in range(NB):
            distance1[i] = pdist([list(Reference), list(BOX_miu[i])], 'euclidean')
            distance2[i] = cp.sqrt(pdist([list(Reference), list(BOX_miu[i])], 'cosine'))
        
        Chessblocak_typicality = []
        for i in range(NB):
            if distance1[i]<n*interval1 and distance2[i]<n*interval2:
                Chessblocak_typicality.append(BOXMT[i])
        if max(Chessblocak_typicality) == BOXMT[ii]:
            Centers.append(Reference)
            ModeNumber += 1
    
    return Centers,ModeNumber

In [13]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target

Icput = {'GridSize':30, 'StaticData':X, 'DistanceType': 'euclidean'}
Mode = 'Offiline'

In [14]:
%time
#def SelfOrganisedDirectionAwareDataPartitioning(Icput, Mode):
if Mode == 'Offline':
    data = Icput['StaticData']
    L, W = data.shape
    N = Icput['GridSize']
    distancetype = Icput['DistanceType']
    X1, AvD1, AvD2, grid_trad, grid_angl = grid_set(data,N)
    GD, Uniquesample, Frequency = Globaldensity_Calculator(data, distancetype)
    BOX,BOX_miu,BOX_X,BOX_S,BOXMT,NB = chessboard_division(Uniquesample,GD,grid_trad,grid_angl, distancetype)
    Center,ModeNumber = ChessBoard_PeakIdentification(BOX_miu,BOXMT,NB,grid_trad,grid_angl, distancetype)
    Members,Membernumber,Membership,IDX = cloud_member_recruitment(ModeNumber,Center,data,grid_trad,grid_angl, distancetype)
        
    Boxparameter = {'BOX': BOX,
                'BOX_miu': BOX_miu,
                'BOX_S': BOX_S,
                'NB': NB,
                'XM': X1,
                'L': L,
                'AvM': AvD1,
                'AvA': AvD2,
                'GridSize': N}
        
if Mode == 'Evolving':
    distancetype = Icput['DistanceType']
    Data2 = Icput['StreamingData']
    data = Icput['AllData']
    Boxparameter = Icput['SystemParams']
    BOX = Boxparameter['BOX']
    BOX_miu = Boxparameter['BOX_miu']
    BOX_S = Boxparameter['BOX_S']
    XM = Boxparameter['XM']
    AvM = Boxparameter['AvM']
    AvA = Boxparameter ['AvA']
    N = Boxparameter ['GridSize']
    NB = Boxparameter ['NB']
    L1 = Boxparameter ['L']
    L2, _ = Data2.shape
        
    for k in range(L2):
        XM, AvM, AvA = data_standardization(Data2[k,:], XM, AvM, AvA, k+L1)
        interval1 = cp.sqrt(2*(XM-cp.sum(cp.power(AvM,2))))/N
        interval2 = cp.sqrt(1-cp.sum(cp.power(AvA,2)))/N
        BOX, BOX_miu, BOX_S, NB = Chessboard_online_division(Data2[k,:], BOX, BOX_miu, BOX_S, NB, interval1, interval2)
        BOX,BOX_miu,BOX_S,NB = Chessboard_online_merge(BOX,BOX_miu,BOX_S,NB,interval1,interval2)
        
    BOXG = Chessboard_globaldensity(BOX_miu,BOX_S,NB)
    Center, ModeNumber = ChessBoard_online_projection(BOX_miu,BOXG,NB,interval1,interval2)
    Members, Membernumber, _, IDX = cloud_member_recruitment(ModeNumber, Center, data, interval1, interval2, distancetype)
        
    Boxparameter['BOX']=BOX
    Boxparameter['BOX_miu']=BOX_miu
    Boxparameter['BOX_S']=BOX_S
    Boxparameter['NB']=NB
    Boxparameter['L']=L1+L2
    Boxparameter['AvM']=AvM
    Boxparameter['AvA']=AvA
        
    
#Output = {'C': Center,
#              'IDX': IDX,
#              'SystemParams': Boxparameter,
#              'DistanceType': distancetype}
#   return Output

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 15 µs
